# Flow

## df -> clean df -> split train, test -> encoded dict -> pd encoded -> categorical train, test

In [ ]:
!pip install transformers
import pandas as pd

from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

from tensorflow.keras.utils import to_categorical

import transformers

import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense

In [ ]:
df = pd.read_csv ('gdrive/MyDrive/Colab Notebooks/Sentiment Analysis/tweet_emotions.csv') # loading the dataset

# df = df[:500] # making the dataset short

df.drop(df[(df['sentiment'] == 'empty')].index, inplace=True) # removing empty sentiments
df.drop('tweet_id', inplace = True, axis = 1) # removing tweet_id column

df.rename(columns = {'content':'sentence'}, inplace = True)

df.replace(to_replace = ['fun', 'happiness'], value = "party", inplace = True) # fun, happiness <-> party
df.replace(to_replace = ['enthusiasm', 'surprise', 'love'], value = "happy", inplace = True) # enthusiasm, surprise, love <-> happy
df.replace(to_replace = ['sadness', 'anger', 'hate'], value = "sad", inplace = True) # sadness, anger <-> sad
df.replace(to_replace = ['boredom', 'worry'], value = "chill", inplace = True) # boredom, worry <-> chill
df.replace(to_replace = ['neutral', 'relief'], value = "normal", inplace = True) # neutral, relief <-> normal

df.dropna(inplace=True) # removing empty values

In [ ]:
df_train = df[:30000]
df_test = df[30000:]

In [ ]:
df_train

,sentiment,sentence
1,sad,Layin n bed with a headache ughhhh...waitin o...
2,sad,Funeral ceremony...gloomy friday...
3,happy,wants to hang out with friends SOON!
4,normal,@dannycastillo We want to trade with someone w...
5,chill,Re-pinging @ghostridah14: why didn't you go to...
...,...,...
30670,party,good morning everbody!
30671,chill,@sweetsheilx Same to your mom too pril
30672,party,after prom party tonight
30673,party,you are so right! @Adrianmw1


# sentiment -> number -> matrix

In [ ]:
# encoded_dict = {'fun':0, 'happiness':1, 'enthusiasm':2, 'surprise':3, 'love':4, 'sadness':5, 'anger':6, 'boredom':7, 'worry':8, 'neutral':9, 'relief':10, 'hate':10}
encoded_dict = {'party':0, 'happy':1, 'sad':2, 'chill':3, 'normal':4}
df_train['sentiment'] = df_train.sentiment.map(encoded_dict)
df_test['sentiment'] = df_test.sentiment.map(encoded_dict)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
df_train['sentiment']

1        2
2        2
3        1
4        4
5        3
        ..
30670    0
30671    3
30672    0
30673    0
30674    0
Name: sentiment, Length: 30000, dtype: int64

In [ ]:
# converting our integer coded Sentiment column into categorical data(matrix)

y_train = to_categorical(df_train.sentiment)
y_test = to_categorical(df_test.sentiment)

In [ ]:
y_train

array([[0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)

In [ ]:
y_test

array([[0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.]], dtype=float32)

In [ ]:
df_train['sentiment']
df_test['sentiment']

30675    4
30676    1
30677    0
30678    1
30679    1
        ..
39995    4
39996    1
39997    1
39998    0
39999    1
Name: sentiment, Length: 9173, dtype: int64

In [ ]:
from transformers import AutoTokenizer,TFBertModel
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert = TFBertModel.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
x_train = tokenizer(
    text=df_train.sentence.tolist(),
    add_special_tokens=True,
    max_length=70,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
x_test = tokenizer(
    text=df_test.sentence.tolist(),
    add_special_tokens=True,
    max_length=70,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [ ]:
input_ids = x_train['input_ids']
attention_mask = x_train['attention_mask']
attention_mask.shape

TensorShape([30000, 70])

In [247]:
# max_len = 70

input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
embeddings = bert(input_ids,attention_mask = input_mask)[0] 

out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32,activation = 'relu')(out)

y = Dense(5,activation = 'sigmoid')(out)
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

In [ ]:
optimizer = Adam(
    learning_rate=5e-05, # this learning rate is for bert model , taken from huggingface website 
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)
# Set loss and metrics
loss = CategoricalCrossentropy(from_logits = True)
metric = CategoricalAccuracy('balanced_accuracy'),
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

In [ ]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 70)]         0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 70)]         0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 70,                                          

In [ ]:
train_history = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']}, 
    y = y_train,
    validation_data = (
    {'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']}, y_test
    ),
  epochs=1,
    batch_size=50
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


KeyboardInterrupt: ignored

In [ ]:
model.evaluate(x_test)

ValueError: ignored

In [ ]:
predicted_raw = model.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})
predicted_raw[0]

array([0.25014728, 0.28680623, 0.47626507, 0.3271343 , 0.37236714,
       0.6720892 , 0.64239943, 0.20607114, 0.85951215, 0.5105293 ,
       0.8833815 , 0.6183316 , 0.9007815 ], dtype=float32)

In [ ]:
y_predicted = np.argmax(predicted_raw, axis = 1)
y_true = df_test.sentiment

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_predicted))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00         4
           2       0.00      0.00      0.00        11
           3       0.00      0.00      0.00         4
           4       0.00      0.00      0.00         7
           5       0.00      0.00      0.00        13
           6       0.00      0.00      0.00        32
           7       0.00      0.00      0.00        11
           8       0.58      0.17      0.26       108
           9       0.00      0.00      0.00         6
          10       0.24      0.50      0.33       109
          11       0.00      0.00      0.00        19
          12       0.39      0.61      0.48       125

    accuracy                           0.33       450
   macro avg       0.09      0.10      0.08       450
weighted avg       0.31      0.33      0.27       450



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
texts = input(str('input the text'))
x_val = tokenizer(
    text=texts,
    add_special_tokens=True,
    max_length=70,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True) 
validation = model.predict({'input_ids':x_val['input_ids'],'attention_mask':x_val['attention_mask']})*100
for key , value in zip(encoded_dict.keys(),validation[0]):
    print(key,value)

input the texti'm bored
anger 28.347307
boredom 34.169758
empty 43.236046
enthusiasm 32.30465
fun 36.532295
happiness 61.646385
hate 58.707577
love 21.380016
neutral 85.19072
relief 53.5706
sadness 86.74858
surprise 59.94938
worry 88.97685
